In [2]:
import pandas as pd
file_path = 'Fixed_news_dataset.csv' 

try:
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # In ra 5 dòng đầu để kiểm tra
    print("Đã đọc file thành công với encoding UTF-8:")
    print(df.head())

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")
    print("Hãy kiểm tra lại tên file hoặc đường dẫn.")

Đã đọc file thành công với encoding UTF-8:
   Unnamed: 0.1  Unnamed: 0      id      author  \
0             0           0  218270         NaN   
1             1           1  218269  doãn hằng    
2             2           2  218268         NaN   
3             3           3  218267         NaN   
4             4           4  218266      mi lan   

                                             content  picture_count  \
0  Chiều 31/7, Công an tỉnh Thừa Thiên - Huế thôn...              3   
1  Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...              1   
2  Kết thi nghiệp THPT 2022 trung bình môn toán, ...              3   
3  Thống đốc Kentucky Andy Beshear hôm 31/7 đợt m...              1   
4  Vụ tai nạn giao thông liên hoàn phố đi Tam Bạc...             12   

   processed        source                                              title  \
0          0     docbao.vn  Tên cướp tiệm vàng Huế đại uý công an, công tá...   
1          0        vtc.vn                   Bỏ mạng 5G, Nga 

In [5]:
# 1. Chọn ra 2 cột cần thiết
df = df[['content', 'topic']]

# 2. Xóa các dòng có giá trị bị thiếu (nếu có)
df.dropna(inplace=True)

# In ra để xem kết quả
print("Dữ liệu sau khi chỉ giữ lại 2 cột và xóa dòng thiếu:")
print(df.head())

Dữ liệu sau khi chỉ giữ lại 2 cột và xóa dòng thiếu:
                                             content                topic
0  Chiều 31/7, Công an tỉnh Thừa Thiên - Huế thôn...            Pháp luật
1  Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...  Sức khỏe - Đời sống
2  Kết thi nghiệp THPT 2022 trung bình môn toán, ...             Giáo dục
3  Thống đốc Kentucky Andy Beshear hôm 31/7 đợt m...             Thế giới
4  Vụ tai nạn giao thông liên hoàn phố đi Tam Bạc...              Thời sự


In [6]:
import re
from pyvi import ViTokenizer

# --- Tải danh sách stopwords ---
stopwords_path = 'vietnamese-stopwords.txt'
try:
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file stopwords tại '{stopwords_path}'.")
    print("Hãy chắc chắn bạn đã tải file và đặt đúng vào thư mục dự án.")
    stopwords = [] # Dùng danh sách rỗng nếu không tìm thấy file

# --- Xây dựng hàm tiền xử lý ---
def preprocess_text(text):
    # 1. Chuyển thành chữ thường
    text = text.lower()
    
    # 2. Xóa các ký tự đặc biệt, số, và dấu câu nhưng giữ lại tiếng Việt
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ]', ' ', text)
    
    # 3. Tách từ
    text = ViTokenizer.tokenize(text)
    
    # 4. Loại bỏ stopwords
    words = text.split()
    words = [word for word in words if word not in stopwords]
    text = ' '.join(words)
    
    return text

# In thử kết quả của hàm với một câu mẫu
sample_text = "Chào mừng bạn đến với Machine Learning, hôm nay trời đẹp quá!!!"
processed_sample = preprocess_text(sample_text)
print(f"Câu gốc: '{sample_text}'")
print(f"Câu sau khi xử lý: '{processed_sample}'")

Câu gốc: 'Chào mừng bạn đến với Machine Learning, hôm nay trời đẹp quá!!!'
Câu sau khi xử lý: 'chào_mừng machine learning hôm_nay trời đẹp'


In [7]:
# Áp dụng hàm preprocess_text cho cột 'content'
print("Bắt đầu quá trình tiền xử lý...")
df['cleaned_content'] = df['content'].apply(preprocess_text)
print("Hoàn thành!")

# Xem kết quả sau khi có thêm cột mới 'cleaned_content'
print(df.head())

Bắt đầu quá trình tiền xử lý...
Hoàn thành!
                                             content                topic  \
0  Chiều 31/7, Công an tỉnh Thừa Thiên - Huế thôn...            Pháp luật   
1  Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...  Sức khỏe - Đời sống   
2  Kết thi nghiệp THPT 2022 trung bình môn toán, ...             Giáo dục   
3  Thống đốc Kentucky Andy Beshear hôm 31/7 đợt m...             Thế giới   
4  Vụ tai nạn giao thông liên hoàn phố đi Tam Bạc...              Thời sự   

                                     cleaned_content  
0  chiều 31 7 công_an tỉnh thừa thiên huế thông b...  
1  thứ_trưởng phát_triển kỹ_thuật_số truyền_thông...  
2  kết_thi nghiệp thpt 2022 trung_bình môn toán n...  
3  thống_đốc kentucky andy beshear hôm 31 7 đợt m...  
4  vụ tai_nạn giao_thông liên_hoàn phố đi tam bạc...  


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Tách dữ liệu thành X (đặc trưng) và y (nhãn)
X = df['cleaned_content']
y = df['topic']

# 2. Chia dữ liệu thành tập train và test (tỷ lệ 80/20)
# random_state=42 giúp kết quả chia luôn giống nhau mỗi lần chạy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Khởi tạo và áp dụng TF-IDF
# Khởi tạo TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Học từ vựng từ tập train và biến đổi X_train thành ma trận TF-IDF
print("Bắt đầu biến đổi dữ liệu train bằng TF-IDF...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Chỉ biến đổi X_test dựa trên từ vựng đã học từ tập train
print("Bắt đầu biến đổi dữ liệu test...")
X_test_tfidf = tfidf_vectorizer.transform(X_test)
print("Hoàn thành!")

# 4. In ra kích thước của ma trận kết quả
print(f"Kích thước của ma trận train (số tài liệu, số từ vựng): {X_train_tfidf.shape}")
print(f"Kích thước của ma trận test (số tài liệu, số từ vựng): {X_test_tfidf.shape}")

Bắt đầu biến đổi dữ liệu train bằng TF-IDF...
Bắt đầu biến đổi dữ liệu test...
Hoàn thành!
Kích thước của ma trận train (số tài liệu, số từ vựng): (128974, 259302)
Kích thước của ma trận test (số tài liệu, số từ vựng): (32244, 259302)


In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# 1. Khởi tạo và Huấn luyện mô hình
print("Bắt đầu huấn luyện mô hình Naive Bayes...")
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)
print("Đã huấn luyện xong!")

# 2. Dự đoán trên tập test
print("Bắt đầu dự đoán trên tập test...")
y_pred = model.predict(X_test_tfidf)
print("Đã dự đoán xong!")

# 3. Đánh giá mô hình
# Tính toán độ chính xác tổng thể
accuracy = accuracy_score(y_test, y_pred)
print("-" * 30)
print(f"🎉 Độ chính xác (Accuracy) của mô hình là: {accuracy * 100:.2f}%")
print("-" * 30)

# In ra báo cáo chi tiết cho từng thể loại
print("Báo cáo phân loại chi tiết (Classification Report):")
print(classification_report(y_test, y_pred))

Bắt đầu huấn luyện mô hình Naive Bayes...
Đã huấn luyện xong!
Bắt đầu dự đoán trên tập test...
Đã dự đoán xong!
------------------------------
🎉 Độ chính xác (Accuracy) của mô hình là: 64.75%
------------------------------
Báo cáo phân loại chi tiết (Classification Report):
                        precision    recall  f1-score   support

               Bạn đọc       0.00      0.00      0.00       344
          Bất động sản       0.00      0.00      0.00       303
             Chính trị       1.00      0.01      0.01       779
             Công nghệ       0.88      0.11      0.19       837
              Giáo dục       0.84      0.52      0.64      1539
Kinh doanh - Tài chính       0.56      0.63      0.60      2066
               Kinh tế       0.00      0.00      0.00      1036
           Pháp luật       0.00      0.00      0.00         1
             Pháp luật       0.76      0.76      0.76      2121
            Quốc phòng       0.00      0.00      0.00       198
   Sức khỏe - Đời số

D:\Python\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Python\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Python\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
